In [3]:
import torch
import torch.nn as nn 
from transformers import BlipProcessor, BlipForConditionalGeneration, BertTokenizer, BertForQuestionAnswering
from flask import Flask, request, jsonify, render_template
import os
from PIL import Image

In [6]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

img_path = "/home/naren/Documents/YoloChat/horse.jpg"
results = model(img_path)
detected_objects = results.pandas().xyxy[0]['name'].tolist()
print("Detected Objects:", detected_objects)

Using cache found in /home/naren/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-1-5 Python-3.12.3 torch-2.5.1+cu124 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Detected Objects: ['horse']


/home/naren/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


In [7]:
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")


img = Image.open(img_path)
inputs = processor(img, return_tensors="pt")
outputs = model.generate(**inputs)

# Decode the output
caption = processor.decode(outputs[0], skip_special_tokens=True)
print("Generated Caption:", caption)

preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

Generated Caption: a black horse running through a field


In [10]:
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

query = "Is the horse middle of running?"
inputs = tokenizer.encode_plus(query, caption, return_tensors="pt")


start_scores, end_scores = model(**inputs, return_dict=False)
start_idx = torch.argmax(start_scores)
end_idx = torch.argmax(end_scores) + 1

answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][start_idx:end_idx]))
print("Answer:", answer)

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Answer: black horse running through a field


In [11]:
app = Flask(__name__)


yolo_model = torch.hub.load('ultralytics/yolov5', 'yolov5s')
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

@app.route('/')
def index():
    return render_template('index.html')


@app.route('/generate_caption', methods=['POST'])
def generate_caption():
    if 'image' not in request.files:
        return jsonify({'error': 'No image uploaded'})

    
    image = request.files['image']
    image_path = os.path.join('uploads', image.filename)
    image.save(image_path)

   
    results = yolo_model(image_path)
    detected_objects = results.pandas().xyxy[0]['name'].tolist()

    
    img = Image.open(image_path)
    inputs = processor(img, return_tensors="pt")
    outputs = blip_model.generate(**inputs)
    caption = processor.decode(outputs[0], skip_special_tokens=True)

    
    os.remove(image_path)

    return jsonify({'detected_objects': detected_objects, 'caption': caption})

if __name__ == '__main__':
    
    os.makedirs('uploads', exist_ok=True)
    app.run(debug=True)


Using cache found in /home/naren/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-1-5 Python-3.12.3 torch-2.5.1+cu124 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/naren/miniconda3/envs/venus/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/naren/miniconda3/envs/venus/lib/python3.12/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/home/naren/miniconda3/envs/venus/lib/python3.12/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/naren/miniconda3/envs/venus/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 692, in initialize
    self.init_sockets()
  File "/home/naren/miniconda3/envs/venus/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 331, in init_sockets
   

SystemExit: 1